<a href="https://www.kaggle.com/code/pogscafe/invokeai-notebook?scriptVersionId=151255489" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

### Installing InvokeAI - Required

In [ ]:

import os
import subprocess

env = os.environ.copy()

!pip install 'InvokeAI[xformers]' --use-pep517 --extra-index-url https://download.pytorch.org/whl/cu118 --break-system-packages

!pip install tensorflow
!pip install --no-deps tensorflow-io 
    
# Install openssh for running the UI with RemoteMoe
!mamba install openssh -y
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### 
### Configuration and downloading default models - Required

In [ ]:
!mkdir /kaggle/temp/invokeai
!mkdir /kaggle/temp/invokeai/configs

#@markdown Download only the default model in initial configuration.
#@markdown Checking this prevents running out of space in Colab.

defaultOnly = True #@param {type:"boolean"}
skipWeights = True #@param {type:"boolean"}
skipSupportModels = False #@param {type:"boolean"}
noFullPrecision = True

#@markdown This step usually takes about 2 minutes with only the default model and no weights.

#@markdown You can ignore "File exists" warnings in the output.

cmd = 'invokeai-configure --root_dir /kaggle/temp/invokeai --yes'

if defaultOnly:
  cmd += ' --default_only'

if skipWeights:
  cmd += ' --skip-sd-weights'

if skipSupportModels:
  cmd += ' --skip-support-models'

subprocess.run(cmd, shell=True, env=env)

import fileinput
import os
def find(name, path):
    for root, dirs, files in os.walk(path):
        if name in files:
            return os.path.join(root, name)

if noFullPrecision:
  model_install_file = find('model_install_backend.py', '/opt/conda/lib/')
  print('modifying file ' + str(model_install_file))
  for line in fileinput.input(model_install_file, inplace=True):
    if ('precision = torch_dtype(choose_torch_device())' in line):
       line = line.replace('torch_dtype(choose_torch_device())', 'torch.float16')
    print(line, end='')

### 
### Add the SDXL base model (optional)

In [ ]:
import os.path
from os import path

# Install the SDXL base model
def installSdxl(env):
  installCmd = 'invokeai-model-install --add "stabilityai/stable-diffusion-xl-base-1.0" --root_dir /kaggle/temp/invokeai'
  subprocess.run(installCmd, shell=True, env=env)
  
alreadyInstalled = True
sdxlBaseSubfolderName = ''
modelsPath = '/kaggle/working/stablemodels/'
sdxlBaseSubfolderName = '/stable-diffusion-xl-base-1-0'
workSdxlMainFolder = modelsPath + 'sdxl/main'
if not path.exists(workSdxlMainFolder):
    os.makedirs(workSdxlMainFolder, exist_ok=True)
    alreadyInstalled = False

tempModelsSdxlFolder = '/kaggle/temp/invokeai/models/sdxl/'
tempSdxlMainFolder = tempModelsSdxlFolder + 'main'

subprocess.run('rm -rf ' + tempModelsSdxlFolder, shell=True, env=env)
if path.exists(tempModelsSdxlFolder):
    subprocess.run('rmdir ' + tempModelsSdxlFolder, shell=True, env=env)

if not alreadyInstalled:
    if not path.exists(tempModelsSdxlFolder):
      os.makedirs(tempModelsSdxlFolder, exist_ok=True)
    subprocess.run('ln -s '+workSdxlMainFolder+' '+tempModelsSdxlFolder, shell=True, env=env)
    installSdxl(env)
else:
    if not path.exists(tempSdxlMainFolder):
      os.makedirs(tempSdxlMainFolder, exist_ok=True)
    subprocess.run('ln -s '+workSdxlMainFolder + sdxlBaseSubfolderName+' '+ tempSdxlMainFolder, shell=True, env=env)
    updateModelsYaml = True
    with open('/kaggle/temp/invokeai/configs/models.yaml') as f:
      if 'stable-diffusion-xl-base-1-0' in f.read():
        updateModelsYaml = False
    if updateModelsYaml:
      with open('/kaggle/temp/invokeai/configs/models.yaml', 'a') as file:
        lines = [
          'sdxl/main/stable-diffusion-xl-base-1-0:\n',
          '  path: sdxl/main/stable-diffusion-xl-base-1-0\n',
          '  description: Stable Diffusion XL base model (12 GB)\n',
          '  variant: normal\n',
          '  format: diffusers\n'
        ]
        file.writelines(lines)

### 
### Add the refiner and vae (optional)

In [ ]:
#This one took about 14 minutes.
#Skip this step if you don't need these models.
!invokeai-model-install --add "stabilityai/stable-diffusion-xl-refiner-1.0" --root_dir /kaggle/temp/invokeai --yes
!invokeai-model-install --add "madebyollin/sdxl-vae-fp16-fix" --root_dir /kaggle/temp/invokeai --yes

### 
### Save images to the working folder (optional)
If file persistance is enabled, images and the database will be saved for future sessions

In [ ]:
import os.path
from os import path

# Linking output images to working folder
outputDrivePath = '/kaggle/working/images/invoke-outputs' #@param {type:"string"}
# Full path to the output folder on Google Drive

saveDatabase = True #@param {type:"boolean"}

if not outputDrivePath.endswith('/'):
  outputDrivePath = outputDrivePath + '/'
imagesDrivePath = outputDrivePath + 'images'
databaseDrivePath = outputDrivePath + 'databases'
if not path.exists(imagesDrivePath):
  os.makedirs(imagesDrivePath, exist_ok=True)


outputsLocalPath = '/kaggle/temp/invokeai/outputs'
imagesLocalPath = '/kaggle/temp/invokeai/outputs/images'

if not path.exists(outputsLocalPath):
  os.makedirs(outputsLocalPath, exist_ok=True)

import datetime

if path.exists(imagesLocalPath):
    cmd = f'mv {imagesLocalPath} {imagesLocalPath}-backup{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}'
    subprocess.run(cmd, shell=True, env=env)

cmd = f'ln -s {imagesDrivePath} {outputsLocalPath}'
subprocess.run(cmd, shell=True, env=env)

# Linking the database
if saveDatabase:
  if not path.exists(databaseDrivePath):
    os.makedirs(databaseDrivePath, exist_ok=True)

  databaseLocalPath = '/kaggle/temp/invokeai/databases'

  cmd = f'mv {databaseLocalPath} {databaseLocalPath}-backup{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}'
  subprocess.run(cmd, shell=True, env=env)

  cmd = f'ln -s {databaseDrivePath} /kaggle/temp/invokeai'
  subprocess.run(cmd, shell=True, env=env)

### 
### Start the WebUI

**Option 1: Starting the Web UI with ngrok**  
* Make sure to put your ngrok token in the Ngrok_token variable. The token can be obtained from https://ngrok.com
* If you have a static domain, put your ngrok domain in the Ngrok_domain variable.
* Wait for the line that says "Uvicorn running on http://127.0.0.1:9090 (Press CTRL+C to quit)" 
* Visit your ngrok URL (either your static domain, or the ngrok url displayed in the output)

In [ ]:
# Option 1: Starting the Web UI with ngrok

Ngrok_token = "" #@param {type:"string"}
# Put your ngrok token here (obtainable from https://ngrok.com)

Ngrok_domain = "" # optional, leave empty if you don't have a domain

# Only works with InvokeAI 3.0.2 and later

!pip install pyngrok

from pyngrok import ngrok, conf
import fileinput
import sys

if Ngrok_token!="":
  ngrok.kill()
  srv=ngrok.connect(9090 , pyngrok_config=conf.PyngrokConfig(auth_token=Ngrok_token),
                    bind_tls=True, domain=Ngrok_domain).public_url
  print(srv)
  get_ipython().system("invokeai-web  --root /kaggle/temp/invokeai/")
else:
  print('An ngrok token is required. You can get one on https://ngrok.com and paste it into the ngrok_token field.')

***   
**Option 2: Starting the Web UI with RemoteMoe**  
* Wait for the line that says "Uvicorn running on http://127.0.0.1:9090 (Press CTRL+C to quit)"   
* Click the link that ends with .remote.moe

In [ ]:
#Option 2: Starting the Web UI with RemoteMoe

# Wait for the line that says "Uvicorn running on http://127.0.0.1:9090 (Press CTRL+C to quit)" 
# and click on the link that ends with .remote.moe

!mkdir  ~/.ssh/
!touch  ~/.ssh/known_hosts
!ssh-keyscan -t rsa remote.moe >> ~/.ssh/known_hosts
!rm /root/.ssh/id_rsa
!ssh-keygen -t rsa -b 4096 -f /root/.ssh/id_rsa -q -N ""
#!ssh -R 80:127.0.0.1:9090 -o StrictHostKeyChecking=no -i /root/.ssh/id_rsa remote.moe 
!invokeai-web --root /kaggle/temp/invokeai/ & ssh -R 80:127.0.0.1:9090 -o StrictHostKeyChecking=no -i /root/.ssh/id_rsa remote.moe 

***   
**Option 3: Starting the Web UI with Localtunnel**  
* Copy the IP address shown in the output above the line "your url is: https://some-random-words.loca.lt"  
* Wait for the line that says "Uvicorn running on http://127.0.0.1:9090 (Press CTRL+C to quit)"   
* Click the localtunnel url and paste the IP you copied earlier to the "Endpoint IP" text field


In [ ]:
# Option 3: Starting the Web UI with Localtunnel
# Warning: Localtunnel has been down recently. If the .loca.lt link doesn't show up in the output
# please use one of the other two options

%cd /kaggle/temp/invokeai/
!npm install -g localtunnel

#@markdown Copy the IP address shown in the output above the line
#@markdown "your url is: https://some-random-words.loca.lt"
!wget -q -O - ipv4.icanhazip.com

#@markdown Wait for the line that says "Uvicorn running on http://127.0.0.1:9090 (Press CTRL+C to quit)"

#@markdown Click the localtunnel url and paste the IP you copied earlier to the "Endpoint IP" text field
!lt --port 9090 --local_https False & invokeai-web  --root /kaggle/temp/invokeai/ --ignore_missing_core_models 

#@markdown If the UI shows a red dot that says 'disconnected' when hovered in the upper
#@markdown right corner and the Invoke button is disabled, change 'https' to 'http'
#@markdown in the browser's address bar and press enter.
#@markdown When the page reloads, the UI should work properly.


In [ ]:
!pwd
